In [6]:
# Merge data
customer_transactions = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

In [7]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

customers = pd.read_csv(r"C:\Users\jaswa\Desktop\Zeotap\Customers.csv")
products = pd.read_csv(r"C:\Users\jaswa\Desktop\Zeotap\Products.csv")
transactions = pd.read_csv(r"C:\Users\jaswa\Desktop\Zeotap\Transactions.csv")

customer_transactions = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')
features = customer_transactions.groupby('CustomerID').agg({
    'Quantity': 'sum',
    'TotalValue': 'sum'
}).reset_index()

features['AveragePrice'] = features['TotalValue'] / features['Quantity']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features[['AveragePrice', 'Quantity', 'TotalValue']])
similarity_matrix = cosine_similarity(scaled_features)
similarity_df = pd.DataFrame(similarity_matrix, index=features['CustomerID'], columns=features['CustomerID'])

In [8]:
lookalikes = {}
for customer in features['CustomerID'][:20]:
    similar_customers = similarity_df[customer].nlargest(4).index[1:]  # Exclude self
    scores = similarity_df[customer].nlargest(4).values[1:]  # Exclude self
    lookalikes[customer] = list(zip(similar_customers, scores))

lookalike_data = []
for customer, similar in lookalikes.items():
    row = [customer]  # Start with the customer ID
    for sim in similar:
        row.append(sim[0])  # Lookalike ID
        row.append(sim[1])  # Similarity Score
    # Fill with None if there are less than 3 lookalikes
    while len(row) < 7:  # We want 1 customer ID + 3 lookalikes + 3 scores = 7 elements
        row.append(None)
    lookalike_data.append(row)

lookalike_df = pd.DataFrame(lookalike_data, columns=['CustomerID', 'Lookalike1', 'Score1', 'Lookalike2', 'Score2', 'Lookalike3', 'Score3'])
lookalike_df.to_csv(r"C:\Users\jaswa\Desktop\Zeotap\Lookalike.csv", index=False)